# Know Error
🟩 The bot tends to genrate multiple responese for a single reply (fixed)

🟩 Genrates multiple responses while working with media files (fixed)

🟩 Is able to remember images only time it is uploaded.(fixed)

🟩 Improving the video responses(Direct function added by google for video file)

🟩 Finding a way to keep the converstion for the videos conversation

🟩 If the user sends a gif through the inbuilt gif section, it's html page is downloaded.(fixed)

[The gif is converted to a mp4 file. If your gif is shorter then it will treat it as a still photo as gemini sees videos with one frame per seconds, which might lead to wrong interpretation of the gif]

🟩 Gifs from the discord keyboard(tenor), doesn't get interpretted correct(because different gifs get donwloaded) (fixed)

 !check_token doesn't  work for direct messages

# Goal
1. Fix the 2000 char limit. ✅
2. Saving the chats. ✅
3. Only reply when tagged or replied ✅
4. Consinent chats ✅
5. Image input ✅
6. Mutimedia input ✅
7. Webhook

# d_bot
This notebook lets you run a discord bot powered by google gemini api, with conversation memory which includes audio, video, and images.

You can type **"!check_token"** to check how many tokens are being used.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1OQGPc2CsYpnBhNfKEl0O0cDT6uXAcQ8g?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  
</table>


In [ ]:
#@title <b>v-- If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or your session might get disconnected
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [1]:
# @title Mount your google drive
from google.colab import drive
from IPython.display import clear_output
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Install requirments

In [2]:
!pip install -q -U google-generativeai
!pip install Discord
!pip install python-magic
!pip install nest_asyncio
!pip install moviepy
!pip install pytz
clear_output()
print("Success")

Success


Step 2: Get the api key

In [3]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# @title Step 2.5: List available models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

print("Now select any one of the model and paste it in the 'model_name' below")

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827
Now select any one of the model and paste it in the 'model_name' below


In [5]:
# Set the event listener for the dropdown change
# @title Model configuration
text_generation_config = {
    "temperature": 1.35,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

# Initial prompt
system_instruction = "Remember that you have the power of python to solve logical question if possible, don't forgot to try. When you the see the user message in the following format = ([string], [number]): {message content}. It means the conversation is happening in a server in discord. The string represents the username of the of the user who have sent the message and the number is the user id of the user.  Multiple people can interact during this, make sure too act accordingly. If you don't see this format and just see this format = (number) it means they are talking to you in dm, so act accordingly." # @param {type:"string"}
model_name = "models/gemini-1.5-pro-latest" # @param {type:"string"}

# Create the model using the selected model name from the dropdown
model = genai.GenerativeModel(model_name = model_name, generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safety_settings, tools="code_execution")

In [12]:
# @title Functions
#Rembeer not to copy the timezone code
from discord import app_commands,Message
import time
from random import choice, randint
import os
import requests
from urllib.parse import urlparse, unquote
import re
import cv2
import shutil
import mimetypes
import magic
import json
import pickle
from datetime import datetime, timedelta
import moviepy.editor as mp
from bs4 import BeautifulSoup
from google.api_core.exceptions import GoogleAPIError

def upload_to_gemini(path, mime_type=None):
    """Uploads the given file to Gemini."""
    file = genai.upload_file(path, mime_type=mime_type)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    return file

def wait_for_files_active(files):
    """Waits for the given files to be active."""
    print("Waiting for file processing...")
    for name in (file.name for file in files):
        file = genai.get_file(name)
        while file.state.name == "PROCESSING":
            print(".", end="", flush=True)
            time.sleep(10)
            file = genai.get_file(name)
        if file.state.name != "ACTIVE":
            raise Exception(f"File {file.name} failed to process")
    print("...all files ready")
    print()

def download_file(attachment_link: str, user_id: str) -> tuple:
    """
    Downloads the file, determines its type, and renames it with the correct extension.
    """
    try:
        # Parse the URL to get the file name without the extension
        parsed_link = urlparse(unquote(attachment_link))
        path = parsed_link.path
        original_filename = os.path.basename(path).split('?')[0]

        # Create a temporary filename with the user_id as prefix and no extension
        temp_filename = f'file_{user_id}'
        temp_filename_no_ext = temp_filename.rsplit('.', 1)[0]

        # Download the file
        response = requests.get(attachment_link)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Save the file without an extension
        with open(temp_filename_no_ext, 'wb') as f:
            f.write(response.content)

        print(f"File downloaded successfully: {temp_filename_no_ext}")

        # Determine the file type and the correct extension
        mime_type = determine_file_type(temp_filename_no_ext)
        extension = mimetypes.guess_extension(mime_type) or '.bin'  # Default to .bin if unknown

        # Check if the downloaded file is actually a GIF
        if mime_type == 'text/html' and 'tenor.com' in parsed_link.netloc:
            media_url = extract_media_url_from_html(temp_filename_no_ext)
            if media_url:
                os.remove(temp_filename_no_ext)
                return download_file(media_url, user_id)
            else:
                raise ValueError("Unable to extract media URL from Tenor HTML")

        # Rename the file with the correct extension
        final_filename = f'{temp_filename_no_ext}{extension}'
        os.rename(temp_filename_no_ext, final_filename)
        print(f"File renamed to: {final_filename} with MIME type: {mime_type}")

        return mime_type, final_filename
    except requests.RequestException as e:
        print(f"Failed to download file. Error: {e}")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return

def determine_file_type(filepath: str) -> str:
    """
    Determines the MIME type of a file by reading its contents.
    """
    try:
        mime = magic.Magic(mime=True)
        mime_type = mime.from_file(filepath)
        return mime_type
    except Exception as e:
        print(f"Could not determine file type. Error: {e}")
        return 'application/octet-stream'

# Function to load chat history from a file
def load_chat_history(user_id, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'

    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    if not os.path.exists(full_path):
        with open(full_path, 'wb') as file:
            pickle.dump([], file)

    with open(full_path, 'rb') as file:
        chat_history = pickle.load(file)

    return chat_history

# Function to save the chat history from a file
def save_chat_history(user_id, chat, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'
    with open(full_path, 'wb') as file:
        pickle.dump(chat.history, file)

def save_filetwo(user_id, time_file_path, url):
    file_path = f'{time_file_path}{user_id}_files_metadata.json'

    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        with open(file_path, 'w') as file:
            json.dump([], file)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    new_data = {
        'file_uri': url,
        'timestamp': datetime.now().isoformat()
    }
    data.append(new_data)
    print(file_path)

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
        print("Successful saved the file url and upload time")

def check_expired_files(user_id, time_file_path, history):
    tempoery = []
    chat_history = history
    file_path = f'{time_file_path}{user_id}_files_metadata.json'

    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            json.dump([], file)
        return history

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    current_time = datetime.utcnow()
    expired_files = []

    for entry in data:
        upload_time = datetime.fromisoformat(entry['timestamp'])
        if current_time - upload_time > timedelta(hours=48):
            expired_files.append(entry)

        for dct in expired_files:
            tempoery.append(dct['file_uri'])

        for link in tempoery:
            target_word = (f'{link}')
            chat_history = [entry for entry in chat_history if target_word not in str(entry)]
            print(f'Successfully removed: {target_word}')

            data = [entry for entry in data if entry['file_uri'] != target_word]

            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
                print("Successfully updated the file_metadata.json")

    return chat_history

def extract_media_url_from_html(html_file_path):
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    media_url = None
    for meta in soup.find_all('meta'):
        if meta.get('name') == 'twitter:player:stream':
            media_url = meta.get('content')
            break

    return media_url

async def send_processing_notice(message: Message) -> None:  # Define the function
    """Sends the "Processing message" notice after a short delay."""
    await asyncio.sleep(0.0001)  # Adjust delay as needed
    await message.channel.send(f"<@{(message.author.id)}> ⚠️ The bot is currently processing another request. Please wait a moment.")

class GeminiErrorHandler:
    def __init__(self):
        self.error_messages = {
            400: "⚠️ Error 400: Invalid Argument. Please check your request format and parameters.",
            403: "⚠️ Error 403: Permission Denied. Check your API key and authentication.",
            404: "⚠️ Error 404: Not Found. The requested resource was not found.",
            429: "⚠️ Error 429: Too Many Requests. Please try again later.",
            500: "⚠️ Error 500: Internal Server Error. Please try again later or reduce the input context.",
            503: "⚠️ Error 503: Service Unavailable. Please try again later or switch to a different model.",
            504: "⚠️ Error 504: Deadline Exceeded. Please try again later or set a larger timeout."
        }

    async def handle_error(self, message: Message, error: GoogleAPIError, id_str: str):
        error_code = error.code if hasattr(error, "code") else 500  # Default to 500 if code not available
        error_message = self.error_messages.get(
            error_code,
            f"⚠️ An unexpected error occurred (code {error_code}). Please try again later."
        )
        await send_message(message, error_message, id_str)
        print(f"Error: {error_message} (Details: {error})")


# Create an instance of the error handler
error_handler = GeminiErrorHandler()

# Running The Bot

In [ ]:
import re
import aiohttp
from typing import Final
import os
import discord
from discord import Intents, Client, Message
from google.colab import userdata
import PIL.Image
import nest_asyncio
from datetime import datetime
from google.ai.generativelanguage_v1beta.types import content
from IPython.display import HTML, display, clear_output
import pytz
import asyncio

# STEP 0: LOAD OUR TOKEN FROM SOMEWHERE SAFE
TOKEN: Final[str] = userdata.get('DISCORD_TOKEN')

# STEP 1: BOT SETUP
intents: Intents = Intents.default()
intents.message_content = True
client: Client = Client(intents=intents)
processing_messages = {}

# STEP 2: MESSAGE FUNCTIONALITY
async def send_message(message: Message, response: str, id_str: str) -> None:
    if len(response) <= 2000:
        await message.channel.send(response)
    else:
        # Split using a regex that considers various whitespace characters
        chunks = re.findall(r".{1,2000}(?:\s|$)", response, re.DOTALL)

        for chunk in chunks:
            # Trim any leading/trailing whitespace, including Unicode whitespace
            chunk = chunk.strip()

            # Ensure the chunk is not empty before sending
            if chunk:
                await message.channel.send(chunk)

@client.event
async def on_message(message: Message) -> None:
    global chat_history, history
    if message.author == client.user:
        return

    utc_now = datetime.utcnow()
    spc_timezone = pytz.timezone('Etc/GMT') # Change the timezone
    spc_now = utc_now.replace(tzinfo=pytz.utc).astimezone(spc_timezone)
    timestamp = spc_now.strftime('%Y-%m-%d %H:%M:%S')
    user_message_with_timestamp = f"{timestamp} - {message.author}: {message.content}"

    is_dm = isinstance(message.channel, discord.DMChannel)
    id_str = str(message.author.id) if is_dm else str(message.channel.id)

    is_reply = message.reference is not None and message.reference.resolved.author == client.user

    bot_mentioned = client.user in message.mentions

    url_pattern = re.compile(r'(https?://[^\s]+)')
    urls = url_pattern.findall(message.content)
    Direct_upload = False
    Link_upload = False
    attach_url = None
    custom_path = '/content/drive/MyDrive/Discord_bot/'
    time_file_path = '/content/drive/MyDrive/Discord_bot/Time_files/'
    tempoery = []

    if message.content.strip() == "!check_token":
        history = load_chat_history(id_str, custom_path)
        chat_history = check_expired_files(id_str, time_file_path, history)
        chat = model.start_chat(history=chat_history)
        response = f"{model.count_tokens(chat.history)}"
        await send_message(message, response, id_str)
        return

    if bot_mentioned or is_reply or is_dm:
        username: str = str(message.author)
        if not is_dm:
            user_message_with_timestamp = f"{timestamp} - ({username},[{message.author.id}]): {message.content}"

        if urls:
            attach_url = urls[0]
            Link_upload = True

        if message.attachments:
            for attachment in message.attachments:
                attach_url = attachment.url
                Direct_upload = True
                break

        print(f"({id_str}): {user_message_with_timestamp}")
        history = load_chat_history(id_str, custom_path)
        chat_history = check_expired_files(id_str, time_file_path, history)
        chat = model.start_chat(history=chat_history)

        if is_reply and not is_dm and id_str in processing_messages:
            nest_asyncio.apply()  # Apply nest_asyncio before creating the task
            asyncio.create_task(send_processing_notice(message))
            return

        processing_messages[id_str] = message.id
        await message.add_reaction('🔴')

        try:
            chat = model.start_chat(history=chat_history)

            if Direct_upload or Link_upload:
                format, downloaded_file = download_file(attach_url, id_str)

                if format in ('image/gif'):
                    gif_clip = mp.VideoFileClip(downloaded_file)
                    output_path = f"{downloaded_file.rsplit('.', 1)[0]}.mp4"
                    gif_clip.write_videofile(output_path, codec='libx264')
                    downloaded_file = output_path
                    format = 'video/mp4'

                media_file = [upload_to_gemini(f"{downloaded_file}", mime_type= f"{format}"),]

                wait_for_files_active(media_file)

                save_filetwo(id_str, time_file_path, media_file[0].uri)

                response = chat.send_message([user_message_with_timestamp, media_file[0]])
                response = response.text

                save_chat_history(id_str, chat, custom_path)
                await send_message(message, response, id_str)
                print(f"Bot: {response}")
                Direct_upload = False
                Link_upload = False

            else:
                response = chat.send_message(user_message_with_timestamp)
                response = response.text

                save_chat_history(id_str, chat, custom_path)
                await send_message(message, response, id_str)
                print(f"Bot: {response}")

        except GoogleAPIError as e:
          await error_handler.handle_error(message, e, id_str)

        finally:
            processing_message_id = processing_messages.pop(id_str, None)
            if processing_message_id:
                processing_message = await message.channel.fetch_message(processing_message_id)
                await processing_message.remove_reaction('🔴', client.user)

# STEP 5: MAIN ENTRY POINT
async def main() -> None:
    nest_asyncio.apply()
    await client.start(TOKEN)

# To run in a cloud service like Google Colab, Kaggle, etc.
nest_asyncio.apply()

# Run the main function
await main()

For viewing /editing the chat_history


1. Enter the user/channel id(""THE FULLPATH IS BASED ON THE DEFAULT BEHAVIOUR OF THE PROGRAM, IF YOU HAVE CHANGED IT THEN CHANGE THE FILE  PATH ACCORDINGLY**)
2. Open the file using the fullpath
3. Run the Styling cell
4. Print the chat history by running the "print_chat_history_nicely(chat_history)"
You can view all the messages in this window You can see some numbers associated with them.
5. To modify a message enter the number associated with the message you want to modify.
A text box will appear where you can entre your new message and click on "sumbit" when done. (The new message would be visible in the new window that was created)
6. Now run the cell [Save back the chat history] and a message will tell your history is saved succesfully




In [ ]:
#@title Type the user/channel id
id = 1264073938175004757 # @param {type:"integer"}
fullpath = f"/content/drive/MyDrive/Discord_bot/{id}_chat_history.pkl"

In [ ]:
#@title Opens the chat history
with open(fullpath, 'rb') as file:
    chat_history = pickle.load(file)

In [ ]:
# @title Styling
"""from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json"""

# Define the custom CSS once
custom_css = """
<style>
  .chat-container {
    display: flex;
    flex-direction: column;
    width: 100%;
    padding: 10px;
    border-radius: 5px;
    background-color: #000000;
    margin-top: 10px;
    box-sizing: border-box;
  }

  .chat-message {
    display: flex;
    align-items: flex-start;
    margin-bottom: 5px;
    border-bottom: 1px solid #FFFFFF;
    width: 100%;
    box-sizing: border-box;
  }

  .message-serial {
    font-weight: bold;
    margin-right: 5px;
    width: 30px;
    flex-shrink: 0;
  }

  .message-role {
    font-weight: bold;
    margin-right: 5px;
    width: 80px;
    flex-shrink: 0;
  }

  .message-text {
    flex: 1;
    word-wrap: break-word;
    white-space: pre-wrap;
  }

  .auto-resize-textarea {
    resize: none;
    overflow-y: hidden;
    min-height: 50px;
    max-height: 300px;
  }
</style>

<script>
function autoResize(textarea) {
    textarea.style.height = 'auto';
    textarea.style.height = textarea.scrollHeight + 'px';
}
</script>
"""

def apply_custom_css():
    """Applies the custom CSS and JavaScript."""
    display(HTML(custom_css))

def print_chat_history_nicely(chat_history):
    """Prints chat history using Colab widgets and custom CSS for enhanced formatting,
    including serial numbers for each message.

    Args:
      chat_history: A list of content objects representing the chat history.
    """
    clear_output(wait=True)
    apply_custom_css()

    chat_container_html = "<div class='chat-container'>"

    message_count = 0
    for content_obj in chat_history:
        if isinstance(content_obj, content.Content):
            message_role = content_obj.role
            text_content = content_obj.parts[0].text

            chat_container_html += f"""
            <div class="chat-message">
              <span class="message-serial">{message_count}. </span>
              <span class="message-role">{message_role}:</span>
              <span class="message-text">{text_content}</span>
            </div>
            """
            message_count += 1
        else:
            print("Unexpected content type found in chat history.")

    chat_container_html += "</div>"
    display(HTML(chat_container_html))

    print("Finished printing chat history.")

def display_and_replace_message(content_obj):
    """Displays a selected chat message, takes user input, and replaces the message with the input.
    Also includes a button to copy the displayed message.

    Args:
        content_obj: A content object representing a single chat message.
    """
    if isinstance(content_obj, content.Content):
        text_content = content_obj.parts[0].text
        message_role = content_obj.role

        input_text = widgets.Textarea(
            value='',
            placeholder='Type your message here...',
            description='New Message:',
            layout=widgets.Layout(width='100%', min_height='50px')
        )

        submit_button = widgets.Button(description="Submit")
        copy_button = widgets.Button(description="Copy Message")

        def update_display():
            clear_output(wait=True)
            apply_custom_css()

            message_element = f"""
            <div class="chat-container">
              <div class="chat-message" id="selected-message">
                <span class="message-role">{message_role}:</span>
                <span class="message-text">{text_content}</span>
              </div>
            </div>
            """
            display(HTML(message_element))
            display(widgets.HBox([copy_button, submit_button]))
            display(input_text)

            # Add JavaScript to make the textarea auto-resize
            display(HTML("""
            <script>
            var textarea = document.querySelector('.widget-textarea textarea');
            textarea.classList.add('auto-resize-textarea');
            textarea.setAttribute('onInput', 'autoResize(this)');
            </script>
            """))

        def on_submit_button_clicked(b):
            nonlocal text_content
            new_text_content = input_text.value
            content_obj.parts[0].text = new_text_content
            text_content = new_text_content
            update_display()

        def on_copy_button_clicked(b):
            js_code = f"""
            var textarea = document.createElement('textarea');
            textarea.value = {json.dumps(text_content)};
            document.body.appendChild(textarea);
            textarea.select();
            document.execCommand('copy');
            document.body.removeChild(textarea);
            """
            display(HTML(f"<script>{js_code}</script>"))
            print("Message copied to clipboard!")

        submit_button.on_click(on_submit_button_clicked)
        copy_button.on_click(on_copy_button_clicked)
        update_display()

    else:
        print("Unexpected content type at the specified index.")

# Example usage:
# print_chat_history_nicely(chat_history)
# display_and_replace_message(chat_history[specific_index])

In [ ]:
#@title Prints the chat history on the screen
print_chat_history_nicely(chat_history)

In [ ]:
#@markdown Replace message
selected_message = 6 # @param {type:"integer"}
display_and_replace_message(chat_history[selected_message])

Textarea(value='Red', description='New Message:', layout=Layout(min_height='50px', width='100%'), placeholder=…

##**Save back the chat history**

In [ ]:
with open(fullpath, 'wb') as file:
    pickle.dump(chat_history, file)
    print("Successfully updated the chat_history.pkl")

Successfully updated the chat_history.pkl
